In [49]:
import torch
from torch.autograd import Variable
import numpy
import ctypes

In [ ]:
class CItem(ctypes.Structure):
    _fields_ = [
                 ('value', ctypes.c_float)
                ]

def py_list2c_array(py_list, size):
    rowType = CItem * size
    resultType = ctypes.POINTER(CItem) * size
    result = resultType()
    a = [[0] * size for i in range(size)]
    for i in range(size):
        row = rowType()
        for j in range(size):
            row[j] = CItem()
            a = ctypes.c_float(py_list[i][j])
            row[j].value = a.value
        result[i] = ctypes.cast(row, ctypes.POINTER(CItem))
    return ctypes.cast(result, ctypes.POINTER(ctypes.POINTER(CItem)))

In [ ]:
cLib = ctypes.cdll.LoadLibrary('./tensor_sum.dll')

cLib.tensor_sum.argtypes = [ctypes.POINTER(ctypes.POINTER(CItem))]
cLib.tensor_sum.restype = ctypes.c_float

In [50]:
w1 = Variable(torch.ones(2, 2)*2, requires_grad = True)
w2 = Variable(torch.ones(2, 2)*3, requires_grad = True)
loss = torch.sum(w1 + w2)
print(loss.data)
print(loss.grad_fn)
loss.backward()
print(w1.grad)
print(w2.grad)

tensor(20.)
tensor([[1., 1.],
        [1., 1.]])
tensor([[1., 1.],
        [1., 1.]])


In [52]:
arr1 = w1.cpu().detach().numpy()
arr2 = w2.cpu().detach().numpy()

size = 2
c_array1 = py_list2c_array(arr1, size)
c_array2 = py_list2c_array(arr2, size)

array([[2., 2.],
       [2., 2.]], dtype=float32)

In [ ]:
loss = cLib.tensor_sum(c_array1, c_array2)     # loss = sum(w1,w2)
print(loss)
# loss.backward()
w1.grad